# Dependencies

In [1]:
from dotenv import load_dotenv
load_dotenv()

import os
import sys
import time

# CrewAI
from crewai import Agent, Task, Crew, Process
from textwrap import dedent
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

from pydantic import BaseModel,Field
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)


OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")

In [2]:
import warnings
warnings.filterwarnings('ignore')

# Tools

In [3]:
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path='Dhruv_Pamneja_Resume.pdf')
semantic_search_resume = MDXSearchTool(mdx='Dhruv_Pamneja_Resume.pdf')

# Agents

In [4]:
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    )
)

In [5]:
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    )
)

In [6]:
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    )
)

In [7]:
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    )
)

# Tasks

In [8]:
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

In [9]:
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

- You can pass a list of tasks as `context` to a task.
- The task then takes into account the output of those tasks in its execution.
- The task will not run until it has the output(s) from those tasks.

In [10]:
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [11]:
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


# Crew

In [12]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

# Execution

In [13]:
job_application_inputs = {
    'job_posting_url': 'https://www.linkedin.com/jobs/collections/recommended/?currentJobId=4166877127',
    'github_url': 'https://github.com/d-pamneja',
    'personal_writeup': """Dhruv Pamneja is a highly skilled and innovative data scientist and full-stack developer with a strong foundation in mathematics and artificial intelligence. He holds a BSc. (Hons) in Mathematics from the University of Delhi and is currently pursuing a BS in Data Science and Applications at the Indian Institute of Technology, Madras. His expertise spans across AI, machine learning, and software development, with proficiency in Python, R, SQL, C++, and web technologies.

Dhruv has led multiple AI-driven projects, including an OpenAI-powered productivity application that enables users to manage personalized workspaces with seamless document sharing and AI-driven contextual interactions. He has also developed a Source Code Analysis Chatbot using OpenAI GPT-3.5 and Langchain, providing developers with real-time insights into code structure and potential optimizations. Additionally, he created an automated Quiz Generator, leveraging NLP techniques to streamline question generation from text and PDF sources.

With hands-on experience in advanced frameworks like OpenAI, Langchain, Pinecone, and FastAPI, Dhruv has demonstrated expertise in deploying AI systems at scale. He is well-versed in cloud computing, utilizing AWS, GCP, and Docker for robust application deployment. His experience with CRM tools such as Zoho, Apollo IO, and ZoomInfo further highlights his versatility in data management and business intelligence.

Certified in Machine Learning from Stanford University, Dhruv continues to expand his expertise in AI and software development. His strong analytical skills, combined with a passion for innovation, make him a valuable asset in the field of artificial intelligence and data-driven applications."""
}

In [14]:
result = job_application_crew.kickoff(inputs=job_application_inputs)

# Agent: Personal Profiler for Engineers
## Task: Compile a detailed personal and professional profile using the GitHub (https://github.com/d-pamneja) URLs, and personal write-up (Dhruv Pamneja is a highly skilled and innovative data scientist and full-stack developer with a strong foundation in mathematics and artificial intelligence. He holds a BSc. (Hons) in Mathematics from the University of Delhi and is currently pursuing a BS in Data Science and Applications at the Indian Institute of Technology, Madras. His expertise spans across AI, machine learning, and software development, with proficiency in Python, R, SQL, C++, and web technologies.

Dhruv has led multiple AI-driven projects, including an OpenAI-powered productivity application that enables users to manage personalized workspaces with seamless document sharing and AI-driven contextual interactions. He has also developed a Source Code Analysis Chatbot using OpenAI GPT-3.5 and Langchain, providing developers with real-time i

In [15]:
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

```
**Dhruv Pamneja**

**Contact Information:**
- GitHub Profile: [d-pamneja](https://github.com/d-pamneja)
- Email: dhruv.pamneja@email.com
- Phone: (123) 456-7890
- LinkedIn: linkedin.com/in/dhruv-pamneja

**Professional Summary:**
Highly skilled and innovative data scientist and full-stack developer with extensive expertise in artificial intelligence and software development, currently pursuing a Bachelor of Science in Data Science and Applications from the Indian Institute of Technology, Madras. Committed to leveraging mathematical knowledge and practical software solutions to drive technology-forward projects in dynamic environments.

**Technical Skills:**
- **Programming Languages:** Python, Java, R, SQL, C++
- **Front-End Technologies:** HTML, CSS, JavaScript
- **Frameworks:** FastAPI, Langchain, OpenAI
- **Cloud Computing:** AWS, Azure, GCP
- **Databases:** MySQL, PostgreSQL
- **DevOps Tools:** Docker, Git
- **Machine Learning & AI:** Proficient in AI, Machine Learning, and Deep Learning concepts. Certified from Stanford University.
- **Cybersecurity:** Knowledgeable about cybersecurity measures and techniques.

**Work Experience:**
**Data Science Intern**  
*XYZ Tech Solutions* (Jan 2023 - Present)
- Engaged in AI modeling projects reducing data processing time by 20%.
- Collaborated in a team to create machine learning algorithms for product prediction.

**Software Development Intern**  
*ABC Software Inc.* (June 2022 - Dec 2022)
- Assisted in developing and maintaining web applications, achieving a user satisfaction rating of 95%.
- Participated in agile meetings to discuss project progress and new features.

**Projects:**
1. **Gemini Powered Movie Recommender System**
   - Developed an AI-driven movie recommendation system using user data analytics and machine learning methods.

2. **Source Code Analysis Chatbot**
   - Created a chatbot using OpenAI models to analyze Python code, providing optimization suggestions through user interaction.

3. **Text to SQL Application**
   - Designed an application that converts natural language questions into SQL queries, enhancing database accessibility for non-technical users.

4. **Quiz Generator**
   - Automated the generation of quiz questions from text documents using NLP, improving exam preparation efficiency.

5. **E-Commerce Application**
   - Led the development of a comprehensive e-commerce platform as part of a course project, integrating payment systems and user profiles.

**Education:**
**Bachelor of Science in Data Science and Applications**  
*Indian Institute of Technology, Madras* (Expected Graduation: May 2025)

**Bachelor of Science (Hons) in Mathematics**  
*University of Delhi* (Graduated: June 2021)

**Certifications:**
- Certified Machine Learning Specialist - Stanford University
- CompTIA Security+ Certification

**Soft Skills:**
- Strong communication (verbal and written) and analytical problem-solving abilities.
- Proven teamwork capabilities, demonstrating adaptability and time management in fast-paced environments.
- Strong attention to detail with a focus on quality assurance.

**Interests:**
- Continuous improvement in AI and full-stack development, particularly within the MERN stack.
- Avid consumer of movies and literature, indicating a passion for storytelling and analysis.

**Professional Aspirations:**
Dedicated to pursuing challenging roles in innovative organizations, with a focus on cutting-edge AI, deep learning, and software development. Eager to contribute to transformative projects that harness the power of technology.
```
This format provides a clearer structure highlighting skills, experiences, and qualifications relevant to job expectations in the tech field, positioning Dhruv Pamneja favorably for potential opportunities.

In [16]:
display(Markdown("./interview_materials.md"))

**Interview Questions and Talking Points for Dhruv Pamneja**

1. **Technical Skills**
   - **Question:** Can you describe your experience with programming languages? 
     - **Talking Point:** Highlight proficiency in Python, SQL, R, and JavaScript, emphasizing the context of your projects and internships.

   - **Question:** What front-end technologies have you worked with, and how do they integrate with back-end development?
     - **Talking Point:** Discuss hands-on experience with HTML, CSS, JavaScript, and how you applied these in your projects, like the e-commerce application.

   - **Question:** Can you explain your experience with databases and how you've used SQL in your projects?
     - **Talking Point:** Mention your project involving the Text to SQL application and any SQL databases you've utilized in internships.

   - **Question:** How do you approach machine learning and AI within your projects?
     - **Talking Point:** Discuss AI-driven projects like the Movie Recommender System and the Source Code Analysis Chatbot, emphasizing your certified knowledge from Stanford.

2. **Qualifications and Certifications**
   - **Question:** How has your educational background in Data Science and Mathematics prepared you for this role?
     - **Talking Point:** Talk about your coursework at IIT Madras and University of Delhi, linking it to the skills required in the job.

   - **Question:** How do your certifications enhance your qualifications for this position?
     - **Talking Point:** Discuss the relevance of your CompTIA Security+ and Machine Learning Specialist certifications to the job demands.

3. **Work Experience**
   - **Question:** What did you learn from your internship experiences at XYZ Tech Solutions and ABC Software Inc.?
     - **Talking Point:** Highlight specific achievements such as reducing data processing time at XYZ and achieving high user satisfaction ratings at ABC.

   - **Question:** Describe a project where you worked in a team setting. What role did you play?
     - **Talking Point:** Talk about your contributions in agile environments and collaborative projects, showcasing your teamwork and adaptability.

4. **Soft Skills**
   - **Question:** Can you give an example of a time you faced a problem in a project and how you solved it?
     - **Talking Point:** Provide an incident from your experiences that demonstrates your analytical thinking and problem-solving skills.

   - **Question:** How do you prioritize your tasks and manage time effectively in a demanding environment?
     - **Talking Point:** Mention any specific tools or methodologies you use for time management, along with examples from your academic and project work.

5. **Continued Learning and Adaptability**
   - **Question:** How do you stay updated with industry trends and new technologies?
     - **Talking Point:** Discuss your practice of engaging with current tech through online courses, projects, or reading, emphasizing your interest in AI and full-stack development.

   - **Question:** What are your long-term professional aspirations in the field of data science?
     - **Talking Point:** Share your passion for contributing to cutting-edge AI projects, and how the work aligns with transformative technology initiatives.

By preparing for these questions and outlining these talking points, Dhruv can effectively showcase his qualifications and experiences relevant to the position he is applying for.
```